# General settings

In [1]:
import os
import pandas as pd
from pandas.tseries.offsets import DateOffset  
import datetime
import numpy as np

In [ ]:
dir_path = os.getcwd()
source = 'source3'
base_path = dir_path + '\\' + source + '\\'

In [ ]:
def read_excel_or_pkl(filename, header):
    path_file_pkl = base_path + 'pickle\\' + filename.replace('xlsx', 'pkl')
    if os.path.exists(path_file_pkl):
        df= pd.read_pickle(path_file_pkl)
    else:
        path_file_xlsx = base_path + filename
        df = pd.read_excel(path_file_xlsx, header)
        df.to_pickle(path_file_pkl)
    return df

In [ ]:
today = pd.Timestamp.today()
current_month = pd.Timestamp(today.year, today.month, 1)
dates_arr = pd.date_range(current_month-DateOffset(months=6), freq='MS', periods=12)

# TURV

### 3.1.1 Average % overtime for the last 3 months

In [ ]:
df_calc_4 = read_excel_or_pkl("calc.xlsx")
df_mis = read_excel_or_pkl("mis.xlsx")
df_nsi = read_excel_or_pkl("nsi.xlsx")
df_result = read_excel_or_pkl("result.xlsx")

In [ ]:
periods_of_medium_overtimedates_arr = pd.date_range(current_month-DateOffset(months=3), freq='MS', periods=3)
df_calc_5 = df_calc_4[(df_calc_4['Period'].isin(periods_of_medium_overtimedates_arr)) & \
                      (df_calc_4['TURV'] != 0)].copy()
df_calc_6 = df_calc_5[['ClinicName', 'Specialization', 'AgeCategory', \
                                 'ExtraHours', 'TURV']]
df_calc_7 = df_calc_6.groupby(['ClinicName', 'Specialization', \
                                          'AgeCategory']).agg({'TURV':sum, 'ExtraHours':sum}).reset_index()

df_calc_7['OvertimePercentLast3Month'] = df_calc_7['ExtraHours'] / df_calc_7['TURV']
df_calc_7.drop(['ExtraHours', 'TURV'], inplace=True, axis=1)
medium_overtimedates = df_calc_7

del df_calc_5
del df_calc_6
del df_calc_7

In [ ]:
df_calc_5 = df_calc_4[(df_calc_4['TURV'] != 0) & \
                      (df_calc_4['Period'] <= current_month.strftime('%Y.%m.%d'))].copy()
df_calc_6 = df_calc_5[['Period', 'ClinicName', 'Specialization', 'AgeCategory', \
                       'TURV', 'StepVC', 'ExtraHours', 'NormWorkTime', \
                       'CountStakePerDay', 'VacationDays', 'NumberCalendarDays']]
df_calc_7 = pd.merge(df_calc_6, df_mis, how="left",
    left_on=['Period', 'ClinicName', 'Specialization', 'AgeCategory'],
    right_on=['Period', 'ClinicName', 'Specialization', 'AgeCategory'], copy=False).fillna(0)
df_calc_8 = df_calc_7[df_calc_7['Visits'] != 0].copy()
df_calc_8.loc[:,'WorkingTime'] = df_calc_8['TURV'] + df_calc_8['ExtraHours']
df_calc_8.loc[:, ['CountStake']] = df_calc_8['CountStakePerDay']
df_calc_8.loc[:, ['DoctorAppointmentTime']] = df_calc_8['Visits']*df_calc_8['StepVC']/60
df_calc_8.loc[:, ['WorkloadTechnical']] = df_calc_8['DoctorAppointmentTime']/(df_calc_8['WorkingTime'])
df_calc_8.loc[:, ['Overtime']] = df_calc_8['ExtraHours']
df_calc_8.loc[:, ['NormWorkingTime']] = df_calc_8['NormWorkTime']
df_calc_8.loc[:, ['OvertimePercentLast3Month']] = df_calc_8['Overtime'] / df_calc_8['TURV']
df_calc_9 = df_calc_8.fillna(0).copy().rename(columns={"NumberCalendarDays": "NumberCalendarDays"})
df_calc_9.drop(['StepVC', 'ExtraHours', 'NormWorkTime'], inplace=True, axis=1)
df_zayavki_v_podbor_old = df_calc_9.fillna(0).copy()

del df_calc_5
del df_calc_6
del df_calc_7
del df_calc_8
del df_calc_9

### 3.2.1 VC

In [ ]:
df_calc_5 = df_calc_4[(df_calc_4['Period'] == current_month.strftime('%Y.%m.%d')) & \
                      (df_calc_4['CountStakePerDay'] != 0).copy()]
df_calc_6 = df_calc_5[['ClinicName', 'Specialization', 'AgeCategory', 'CountStakePerDay']]
df_number_staff = df_calc_6.groupby(['ClinicName', 'Specialization',
                               'AgeCategory']).agg({'CountStakePerDay':sum}).copy().reset_index()
del df_calc_5
del df_calc_6

In [ ]:
df_calc_5 = df_calc_4[(df_calc_4['Period'] > current_month)].copy()
df_calc_6 = df_calc_5[['Period', 'ClinicName', 'Specialization', 'AgeCategory', 'NumberCalendarDays', \
                       'VisitsVC', 'StepVC', 'VacationDaysPeriodеPlan', \
                       'NormWorkTime']]
df_calc_6 = df_calc_6.groupby(['Period', 'ClinicName', 'Specialization', 'AgeCategory',  'NumberCalendarDays', \
                               'VisitsVC', 'StepVC', \
                               'NormWorkTime']).agg({'VacationDaysPeriodеPlan':sum}).copy().reset_index()
df_calc_7 = pd.merge(df_calc_6, medium_overtimedates, how="left",
                     left_on=['ClinicName', 'Specialization', 'AgeCategory'],
                     right_on=['ClinicName', 'Specialization', 'AgeCategory'], copy=False).fillna(0)
df_calc_8 = pd.merge(df_calc_7, df_number_staff, how="left",
                     left_on=['ClinicName', 'Specialization', 'AgeCategory'],
                     right_on=['ClinicName', 'Specialization', 'AgeCategory'], copy=False).fillna(0)
df_calc_8['CountStake'] = df_calc_8['CountStakePerDay']
df_calc_9 = df_calc_8.copy()
df_calc_9.loc[:, ['DoctorAppointmentTime']] = df_calc_9['Visits']*df_calc_9['StepVC']/60
df_zayavki_v_podbor_VC = df_calc_9.fillna(0).copy()

del df_calc_5
del df_calc_6
del df_calc_7
del df_calc_8
del df_calc_9

In [ ]:
df_zayavki_v_podbor = pd.concat([df_zayavki_v_podbor_old, df_zayavki_v_podbor_VC])

del df_zayavki_v_podbor_old
del df_zayavki_v_podbor_VC

### 3.2.3 ESTAFF

In [ ]:
def custom_date_parser(date_str):
    date = datetime.datetime.strptime(date_str, '%d.%m.%Y')
    return date

def search_clinic_by_str(business_unit_str):
    if isinstance(business_unit_str, str) != True:
        return np.nan
    business_unit = np.nan
    list_units = business_unit_str.split('»')
    for i in range(len(list_units)):
        unit = list_units[i].strip()
        if unit.startswith(('OOO',)):
            business_unit = unit
            break
    return business_unit

df_estaff_1 = pd.read_excel(base_path + 'ESTAFF.xlsx', header=0, parse_dates=['Opening date'], \
                          converters={'Final candidate': lambda x: x != ''},
                          date_parser=custom_date_parser, \
                          index_col='Opening date', \
                          dtype={'Days at work':np.uint16, 
                                 'Stage':'category'
                                },
                          usecols = ['Name', 'Assets', 'Subdivision',
                                     'Days at work', 'SizeRate', 'Opening date',
                                     'Final candidate'
                                    ] 
                         )
df_estaff_1['ClinicName'] = df_estaff_1.apply(lambda row: search_clinic_by_str(row['Subdivision']), axis=1)
df_estaff_1 = df_estaff_1[(df_estaff_1['Name'].str.contains(r'\bDoctor')) &
                              (df_estaff_1['ClinicName'].isnull() == False)
                             ].copy().reset_index()
df_estaff_1.loc[df_estaff_1['Assets'].isnull(), ['Assets']] = df_estaff_1['ClinicName']

In [ ]:
df_estaff_nsi_position = read_excel_or_pkl('ESTAFF_NSI_positions.xlsx', 0)
df_estaff_2 = pd.merge(df_estaff_1, df_estaff_nsi_position, how="left", left_on=['Name'],right_on=['Name']) 
df_estaff_2['ReferencePosition_'] = df_estaff_2['Name_NSI'].str.lower()

del df_estaff_nsi_position

In [ ]:
df_estaff_nsi_clinic = read_excel_or_pkl('ESTAFF_NSI_clinic.xlsx', 0)
df_estaff_nsi_clinic_1 = df_estaff_nsi_clinic[df_estaff_nsi_clinic['ParticipateReport'].isnull() == False]

In [ ]:
df_estaff_3 = pd.merge(df_estaff_2, df_estaff_nsi_clinic_1, how="inner",
                       left_on=['Assets'], right_on=['Assets'])
df_estaff_itog = df_estaff_3[['OpeningDate', 'ReferencePosition_', 'AgeCategory', \
                              'ClinicName_NSI', 'SizeRate', 'FinalCandidate']]\
                              .copy().rename(columns={"ClinicName_NSI": "ClinicName"})
del df_estaff_nsi_clinic_1
del df_estaff_2
del df_estaff_3

In [ ]:
df_nsi_for_estaff_init = df_nsi[['ReferencePosition_', 'Specialization']].copy()
df_nsi_for_estaff_init = df_nsi_for_estaff_init[df_nsi_for_estaff_init['Specialization'] != '-']
df_nsi_for_estaff_init['Quantity'] = 1
df_nsi_for_estaff_1 = df_nsi_for_estaff_init.groupby(['ReferencePosition_',
                                               'Specialization']).agg({'Quantity':sum}).copy().reset_index()
df_nsi_for_estaff_2 = df_nsi_for_estaff_1[['ReferencePosition_', 'Quantity']]
df_nsi_for_estaff_2 = df_nsi_for_estaff_2.groupby(['ReferencePosition_']).agg({'Quantity':max})
df_nsi_for_estaff_3 = pd.merge(df_nsi_for_estaff_1, df_nsi_for_estaff_2,
                            how="inner",
                            left_on=['ReferencePosition_', 'Quantity'],
                            right_on=['ReferencePosition_', 'Quantity']) 
df_nsi_for_estaff_4 = df_nsi_for_estaff_3[['ReferencePosition_', 'Specialization']].copy()
df_nsi_for_estaff_5 = pd.merge(df_estaff_itog, df_nsi_for_estaff_4,
                            how="inner",
                            left_on=['ReferencePosition_'],
                            right_on=['ReferencePosition_']) 
df_nsi_for_estaff_5.drop(['ReferencePosition_'], inplace=True, axis=1)

In [ ]:
def insert_time(date):
    return date.replace(day=1)

df_nsi_for_estaff_5['Period'] = df_nsi_for_estaff_5.apply(lambda r: insert_time(r['OpeningDate']), axis=1).copy()
df_nsi_for_estaff_6 = df_nsi_for_estaff_5.rename(columns={'SizeRate': 'SubmittedRequestMonthSub'}).copy()
df_nsi_for_estaff_7 = df_nsi_for_estaff_6.groupby(['Period', 'ClinicName', 'Specialization', 'AgeCategory']) \
                                    .agg({'SubmittedRequestMonthSub':sum}).copy().reset_index()

In [ ]:
df_zayavki_v_podbor_1 = pd.merge(df_zayavki_v_podbor, df_nsi_for_estaff_7, how="left",
                     left_on=['Period', 'ClinicName', 'Specialization', 'AgeCategory'],
                     right_on=['Period', 'ClinicName', 'Specialization', 'AgeCategory'], copy=False)

In [ ]:
del df_nsi_for_estaff_6
del df_nsi_for_estaff_7

In [ ]:
def get_month_of_expected_return_to_work(date, final_candidate):
    date = date + pd.DateOffset(months=1)
    if date < today:
        date = today
        if final_candidate == False:
            date = date + DateOffset(months=1)
    date = date.replace(day=1)
    return date

df_nsi_for_estaff_6 = df_nsi_for_estaff_5[['ClinicName', 'Specialization', 'AgeCategory', 'FinalCandidate',
                                           'SizeRate', 'OpeningDate']].copy().reset_index()
df_nsi_for_estaff_7 = df_nsi_for_estaff_6.rename(
    columns={'SizeRate': 'SubmittedRequestMonthStart'}).copy()
df_nsi_for_estaff_7['Period'] = df_nsi_for_estaff_7.apply(
    lambda r: get_month_of_expected_return_to_work(r['OpeningDate'], r['FinalCandidate']), axis=1)
df_nsi_for_estaff_8 = df_nsi_for_estaff_7.groupby(['Period', 'ClinicName', 'Specialization', 'AgeCategory']) \
    .agg({'SubmittedRequestMonthStart': sum}).copy().reset_index()
df_zayavki_v_podbor_2_0 = pd.merge(df_zayavki_v_podbor_1, df_nsi_for_estaff_8, how="left",
                                 left_on=['Period', 'ClinicName',
                                          'Specialization', 'AgeCategory'],
                                 right_on=['Period', 'ClinicName', 'Specialization', 'AgeCategory'], copy=False)

In [ ]:
max_period = max(dates_arr)
mask = df_zayavki_v_podbor_2_0['SubmittedRequestMonthStart'].isnull() == False
df_zayavki_v_podbor_2_1 = df_zayavki_v_podbor_2_0[mask].copy()
df_zayavki_v_podbor_2_2 = df_zayavki_v_podbor_2_1.copy()
while len(df_zayavki_v_podbor_2_1) > 0:
    df_zayavki_v_podbor_2_1['Period'] = df_zayavki_v_podbor_2_1['Period'] + pd.DateOffset(months=1)
    df_zayavki_v_podbor_2_1 = df_zayavki_v_podbor_2_1[df_zayavki_v_podbor_2_1['Period'] <= max_period]
    df_zayavki_v_podbor_2_2 = pd.concat([df_zayavki_v_podbor_2_2, df_zayavki_v_podbor_2_1])
df_zayavki_v_podbor_2_3 = df_zayavki_v_podbor_2_2.groupby(['Period',
                                                           'ClinicName',
                                                           'Specialization',
                                                           'AgeCategory']) \
                                                 .agg({'SubmittedRequestMonthStart':sum}) \
                                                 .copy() \
                                                 .reset_index()
df_zayavki_v_podbor_2_4 = df_zayavki_v_podbor_2_3.rename(columns={"SubmittedRequestMonthStart":
                                                                  "SubmittedRequestMonthStart_1"}).copy()
df_zayavki_v_podbor_2_5 = pd.merge(df_zayavki_v_podbor_2_0, df_zayavki_v_podbor_2_4,
                                        how="left",
                                        left_on=['Period', 'ClinicName', 'Specialization', 'AgeCategory'],
                                        right_on=['Period', 'ClinicName', 'Specialization', 'AgeCategory'],
                                        copy=False)
mask = df_zayavki_v_podbor_2_5['SubmittedRequestMonthStart_1'].isnull() == False
df_zayavki_v_podbor_2_5.loc[mask, ['CountStake']] = df_zayavki_v_podbor_2_5['CountStake'] + \
                            df_zayavki_v_podbor_2_5['SubmittedRequestMonthStart_1']

df_zayavki_v_podbor_2_5.drop(['SubmittedRequestMonthStart_1'], inplace=True, axis=1)
df_zayavki_v_podbor_2_5.to_excel('df_zayavki_v_podbor_2_5.xlsx')

In [ ]:
df_zayavki_v_podbor_2_5.loc[(df_zayavki_v_podbor_2_5['Period'] > current_month) &
                             (df_zayavki_v_podbor_2_5['CountStake'] > 0), ['TURV']] = \
            df_zayavki_v_podbor_2_5['NormWorkingTime'] * df_zayavki_v_podbor_2_5['CountStake'] * 0.97 - \
            (df_zayavki_v_podbor_2_5['NormWorkingTime'] / df_zayavki_v_podbor_2_5['NumberCalendarDays'] * \
                        df_zayavki_v_podbor_2_5['VacationDays'])

df_zayavki_v_podbor_2_5.loc[:, ['AverageAnnualNormWorkWorkingTimeMonth']] = 20.6*6.6
df_zayavki_v_podbor_2_5.loc[:, ['AverageNumberCalendarDaysMonth']] = 365/12
df_zayavki_v_podbor_2_5.loc[:, ['NormalProcOvertimeDueToRateDifference']] = \
    ((df_zayavki_v_podbor_2_5['AverageAnnualNormWorkWorkingTimeMonth'] / \
         df_zayavki_v_podbor_2_5['AverageNumberCalendarDaysMonth']) / \
     (df_zayavki_v_podbor_2_5['NormWorkingTime'] / \
         df_zayavki_v_podbor_2_5['NumberCalendarDays']) \
    ) - 1
df_zayavki_v_podbor_2_5.loc[df_zayavki_v_podbor_2_5['Period'] > current_month, ['Overtime']] = \
        df_zayavki_v_podbor_2_5['TURV'] * (df_zayavki_v_podbor_2_5['OvertimePercentLast3Month'] + \
            df_zayavki_v_podbor_2_5['NormalProcOvertimeDueToRateDifference'])
df_zayavki_v_podbor_2_5.loc[:, ['WorkingTime']] = df_zayavki_v_podbor_2_5['TURV'] + df_zayavki_v_podbor_2_5['Overtime']
df_zayavki_v_podbor_2_5.loc[df_zayavki_v_podbor_2_5['WorkingTime'] > 0, ['WorkloadTechnical']] = df_zayavki_v_podbor_2_5['DoctorAppointmentTime']/ \
                                                                    df_zayavki_v_podbor_2_5['WorkingTime']

Create table with "RequiredRequestMonthStart"

In [ ]:
def calculation_workload_old_period(df, index, employees):
    appointment_time = df.loc[index,'DoctorAppointmentTime']
    normal_working_hours = df.loc[index,'NormWorkingTime']
    added_employees = employees - df.loc[index,'CountStake']
    working_time = df.loc[index,'WorkingTime']
    workload = appointment_time/(working_time+added_employees*normal_working_hours)
    return workload

def calculation_workload_future_period(df, index, employees):
    appointment_time = df.loc[index,'DoctorAppointmentTime']
    normal_working_hours = df.loc[index,'NormWorkingTime']
    vacation_days = df.loc[index,'VacationDays']
    calendar_days_in_month = df.loc[index,'NumberCalendarDays']
    coefficient = 1 + df.loc[index,'OvertimePercentLast3Month'] + \
                      df.loc[index,'NormalProcOvertimeDueToRateDifference']
    workload = appointment_time/((employees*normal_working_hours*0.97 - 
        (normal_working_hours/calendar_days_in_month*vacation_days))*coefficient)
    return workload

def calculation_workload(df, index, employees):
    if df.loc[index,'Period'] > current_month:
        return calculation_workload_future_period(df, index, employees)
    else:
        return calculation_workload_old_period(df, index, employees)

def calculation_increase_by(df, index, general_increase_by):
    increase_by = 0
    employees_with_applications = df.loc[index,'CountStakeWithRequest']
    workload = calculation_workload(df, index, employees_with_applications + general_increase_by)
    while workload >= 0.7:
        increase_by = increase_by + 0.25
        employees_with_applications_cal = employees_with_applications + \
            general_increase_by + increase_by
        workload = calculation_workload(df, index, employees_with_applications_cal)
    return increase_by

def calculation(df):
    general_increase_by = 0
    for index in df.index:
        increase_by = calculation_increase_by(df, index, general_increase_by)
        general_increase_by = general_increase_by + increase_by
        df.loc[index,'RequiredRequestMonthStart'] = increase_by
        employees_with_applications = df.loc[index,'CountStakeWithRequest']
        df.loc[index,'CountStakeWithRequest'] = employees_with_applications + general_increase_by
    return df

df_zayavki_v_podbor_2 = df_zayavki_v_podbor_2_5.copy()

df_zayavki_v_podbor_2['RequiredRequestMonthStart'] = 0 
df_zayavki_v_podbor_2['CountStakeWithRequest'] = df_zayavki_v_podbor_2['CountStake']
df_zayavki_v_podbor_3 = df_zayavki_v_podbor_2.groupby(by=['ClinicName','Specialization','AgeCategory']).apply(lambda mini_df: calculation(mini_df))

mask = df_zayavki_v_podbor_3['Period'] <= current_month
df_zayavki_v_podbor_3.loc[mask, ['WorkingTimeWithRequest']] = df_zayavki_v_podbor_3['WorkingTime'] + \
                                                                                    (df_zayavki_v_podbor_3['CountStakeWithRequest'] - \
                                                                                         - df_zayavki_v_podbor_3['CountStake']) * \
                                                                                            df_zayavki_v_podbor_3['NormWorkingTime']
mask = df_zayavki_v_podbor_3['Period'] > current_month
df_zayavki_v_podbor_3.loc[mask, ['WorkingTimeWithRequest']] = df_zayavki_v_podbor_3['NormWorkingTime'] * \
                                                                                    df_zayavki_v_podbor_3['CountStakeWithRequest'] * \
                                                                                    0.97 - (df_zayavki_v_podbor_3['NormWorkingTime']/ \
                                                                                            df_zayavki_v_podbor_3['NumberCalendarDays'] * \
                                                                                            df_zayavki_v_podbor_3['VacationDays'])
df_zayavki_v_podbor_3.loc[mask, ['WorkingTimeWithRequest']] = df_zayavki_v_podbor_3['WorkingTimeWithRequest'] * \
                                                                                 (1 + df_zayavki_v_podbor_3['OvertimePercentLast3Month'] + \
                                                                                      df_zayavki_v_podbor_3['NormalProcOvertimeDueToRateDifference'])
mask = df_zayavki_v_podbor_3['RequiredRequestMonthStart'] == 0
df_zayavki_v_podbor_3.loc[mask, ['RequiredRequestMonthStart']] = np.NaN
df_zayavki_v_podbor_4 = df_zayavki_v_podbor_3.copy()
df_zayavki_v_podbor_4.to_excel('df_zayavki_v_podbor_4.xlsx')

In [ ]:
df_zayavki_v_podbor_4_1 = df_zayavki_v_podbor_4[df_zayavki_v_podbor_4['RequiredRequestMonthStart'] != 0].copy()
df_zayavki_v_podbor_4_1['Period'] = df_zayavki_v_podbor_4_1['Period'] - pd.offsets.MonthBegin()
df_zayavki_v_podbor_4_2 = df_zayavki_v_podbor_4_1[['Period', 'ClinicName', 'Specialization', \
                                                   'AgeCategory', \
                                                   'RequiredRequestMonthStart']].copy().reindex()
df_zayavki_v_podbor_4_2 = df_zayavki_v_podbor_4_2.rename(
                                                    columns={'RequiredRequestMonthStart': \
                                                             'RequiredRequestMonthSub'})
df_zayavki_v_podbor_5 = pd.merge(df_zayavki_v_podbor_4, df_zayavki_v_podbor_4_2, how="left",
                                    left_on=['Period', 'ClinicName',
                                        'Specialization', 'AgeCategory'],
                                    right_on=['Period', 'ClinicName', 
                                        'Specialization', 'AgeCategory'], copy=True)
df_zayavki_v_podbor_5.to_excel('df_zayavki_v_podbor_5.xlsx')

In [ ]:
df_zayavki_v_podbor_6 = df_zayavki_v_podbor_5[df_zayavki_v_podbor_5['WorkingTimeWithRequest']>=0]

order = ['Period',
            'ClinicName',
            'AgeCategory',
            'Specialization',
            'TURV',
            'CountStakePerDay',
            'VacationDays',
            'Visits',
            'WorkingTime',
            'CountStake',
            'DoctorAppointmentTime',
            'NormWorkingTime',
            'Overtime',
            'OvertimePercentLast3Month',
            'WorkloadTechnical',
            'SubmittedRequestMonthSub',
            'SubmittedRequestMonthStart',
            'RequiredRequestMonthSub',
            'RequiredRequestMonthStart',
            'CountStakeWithRequest',
            'WorkingTimeWithRequest']
df_result  = df_zayavki_v_podbor_6[order]

# Update BI

In [ ]:
import pyodbc
import urllib
import sqlalchemy as sa
from contextlib import closing

df_result = df_result.round(decimals=4)
for k in df_result.keys():
    df_result.loc[df_result[k] == float('-inf'), k] = None
    df_result.loc[df_result[k] == float('inf'), k] = None

pyodbc.pooling = False
conn_str = (
    r'DRIVER={SQL Server};'
    r'Server=**********;'
    r'Database=**********;'
    r';UID=**********;PWD=**********'+"\\")
quoted_conn_str = urllib.parse.quote_plus(conn_str)
engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted_conn_str))
periods = tuple(set(df_result['Period'].apply(lambda x: x.date().strftime('%Y-%m-%d')).to_list()))
if len(periods) == 1:
    periods = f"('{periods[0]}')"
with closing(engine.connect()) as conn:
    conn.execute(f"""DELETE FROM dim.HrRecruitmentRequest 
                      WHERE Period in {periods} 
                        AND ClinicName in {tuple(set(df_result['ClinicName'].to_list()))}""")
    df_result.to_sql('HrRecruitmentRequest', conn, 'dim', index=False, if_exists='append')
conn.close()
engine.dispose()